# 1. Introdução e Limitações dos Sistemas RAG Convencionais 

1.1. Contextualização da RAG 

A Recuperação Aumentada por Geração (RAG) é uma abordagem que combina modelos de linguagem avançados com sistemas de recuperação de informação para produzir respostas mais precisas e relevantes. Enquanto os modelos de linguagem, como o GPT-4o, Gemini,Llama, Qwen e vários outros, possuem um vasto conhecimento adquirido durante o treinamento, eles podem carecer de informações atualizadas ou específicas de um domínio. A RAG aborda essa limitação ao integrar mecanismos de recuperação que buscam dados externos para complementar a geração de texto. Todos esses assuntos forem vistos na Trilha Fundamentos de RAG!  

Funcionamento Básico da RAG: 

Recuperação: Dada uma consulta ou pergunta, o sistema recupera documentos ou informações relevantes de uma base de dados externa. 

Geração: Utilizando um modelo de linguagem, a resposta é gerada com base tanto na consulta quanto nos documentos recuperados, garantindo que a resposta seja informada e contextualizada. 

1.2. Desafios dos Sistemas RAG Básicos 

Apesar de seus benefícios, os sistemas RAG convencionais enfrentam diversos desafios: 

1.2.1. Alucinações 

Os modelos de linguagem podem gerar informações não suportadas pelos documentos recuperados, fenômeno conhecido como alucinação. Isso ocorre quando o modelo incorpora dados incorretos ou irrelevantes em suas respostas, comprometendo a confiabilidade do sistema. 

1.2.2. Relevância Limitada na Recuperação 

Métodos de recuperação tradicionais, como BM25 ou TF-IDF, baseiam-se na correspondência de palavras-chave e podem não capturar o significado semântico profundo das consultas. Isso pode levar à recuperação de documentos irrelevantes ou menos úteis. 

1.2.3. Incapacidade de Lidar com Consultas Ambíguas 

Consultas ambíguas ou incompletas podem resultar em respostas inadequadas. Sistemas RAG básicos podem não possuir mecanismos eficientes para desambiguar ou refinar consultas, afetando a qualidade da informação fornecida. 

1.3. Impacto das Limitações na Aplicação Prática 

As limitações mencionadas têm implicações significativas em aplicações do mundo real: 

Confiabilidade em Domínios Críticos: Em áreas como saúde ou finanças, informações incorretas podem ter consequências graves. Alucinações ou falta de precisão podem comprometer a confiança no sistema. 

Experiência do Usuário: Respostas imprecisas ou irrelevantes podem frustrar os usuários, diminuindo a eficácia de assistentes virtuais ou chatbots. 

Adaptação a Domínios Específicos: A incapacidade de lidar com terminologias ou contextos especializados limita a adoção de sistemas RAG em setores especializados.

1.4 Exemplos Práticos 



## Script 1.4.1: Implementação de um Sistema RAG Básico com Recuperação Esparsa (BM25)
Este script demonstra a implementação de um sistema RAG básico que utiliza o algoritmo BM25 para recuperação de documentos e um modelo de linguagem simples para geração de respostas. O objetivo é ilustrar como sistemas RAG convencionais operam e identificar possíveis limitações.

In [2]:
# Instalação das bibliotecas necessárias (execute apenas se ainda não estiverem instaladas)
!pip install -q rank_bm25 transformers pandas numpy nltk

# Importação das bibliotecas necessárias
from rank_bm25 import BM25Okapi
from transformers import pipeline
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Download dos recursos necessários do NLTK (execute apenas uma vez)
nltk.download('punkt')

# Carregamento de um conjunto de documentos de exemplo
documents = [
    "O aprendizado de máquina é uma subárea da inteligência artificial focada em algoritmos.",
    "Redes neurais são modelos computacionais inspirados no cérebro humano.",
    "A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.",
    "Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.",
    "A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas."
]

# Exibição dos documentos
df = pd.DataFrame(documents, columns=['Documentos'])
print(df)

# Definição da consulta do usuário
consulta = "Como os modelos de linguagem podem gerar texto?"

# Tokenização dos documentos
tokenized_documents = [word_tokenize(doc.lower()) for doc in documents]

# Criação do objeto BM25
bm25 = BM25Okapi(tokenized_documents)

# Tokenização da consulta
tokenized_query = word_tokenize(consulta.lower())

# Obtenção das pontuações BM25 para a consulta
doc_scores = bm25.get_scores(tokenized_query)

# Obtenção dos índices dos documentos mais relevantes
top_k = 3  # Número de documentos a serem recuperados
top_n = bm25.get_top_n(tokenized_query, documents, n=top_k)

# Exibição dos documentos recuperados
print("\nDocumentos Recuperados:")
for doc in top_n:
    print(f"- {doc}")

# Uso de um modelo de linguagem para gerar a resposta
generator = pipeline('text-generation', 
                    model='distilgpt2',
                    device='cpu',  # Especifica explicitamente o uso da CPU
                    truncation=True)  # Ativa explicitamente a truncagem

# Concatenando os documentos recuperados
contexto = " ".join(top_n)

# Construção do prompt para o modelo de linguagem
prompt = f"Contexto: {contexto}\nPergunta: {consulta}\nResposta:"

# Geração da resposta
resposta = generator(
    prompt,
    max_new_tokens=50,  # Usa max_new_tokens em vez de max_length
    num_return_sequences=1,
    pad_token_id=generator.tokenizer.eos_token_id,  # Define explicitamente o pad_token_id
    clean_up_tokenization_spaces=True  # Define explicitamente
)

# Exibição da resposta
print("\nResposta Gerada:")
print(resposta[0]['generated_text'])

/home/anderson/miniconda3/envs/RAG/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/anderson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                          Documentos
0  O aprendizado de máquina é uma subárea da inte...
1  Redes neurais são modelos computacionais inspi...
2  A recuperação de informação trata de obter inf...
3  Modelos de linguagem pré-treinados podem gerar...
4  A engenharia de prompt é crucial para orientar...

Documentos Recuperados:
- Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.
- A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas.
- A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.


/home/anderson/miniconda3/envs/RAG/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Resposta Gerada:
Contexto: Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano. A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas. A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.
Pergunta: Como os modelos de linguagem podem gerar texto?
Resposta: Como os modelos de linguagem podem gerar texto?


In [3]:
# Adicione este código após o cálculo dos doc_scores
print("\nPontuações BM25 para cada documento:")
for idx, score in enumerate(doc_scores):
    print(f"Documento {idx}: {score:.4f}")


Pontuações BM25 para cada documento:
Documento 0: 0.2198
Documento 1: 0.2528
Documento 2: 0.3853
Documento 3: 4.4616
Documento 4: 0.8192


Análise das Pontuações BM25

O resultado mostra que o BM25 funcionou corretamente, onde:
O Documento 3 ("Modelos de linguagem pré-treinados podem gerar texto semelhante ao humano.") obteve a maior pontuação (4.4616), o que é esperado já que está mais diretamente relacionado à consulta "Como os modelos de linguagem podem gerar texto?"
O Documento 4 ficou em segundo lugar (0.8192) por mencionar "modelos de linguagem"
Os outros documentos receberam pontuações baixas (<0.4) por terem menor relevância para a consulta
Conclusão: O BM25 foi eficaz em identificar o documento mais relevante para a consulta, demonstrando que a recuperação está funcionando adequadamente.

## Script 1.4.2: Demonstrando Alucinações em Sistemas RAG Convencionais
 Este script ilustra como modelos de linguagem podem gerar alucinações quando não estão adequadamente fundamentados nos documentos recuperados.

In [10]:
# Reutilização dos documentos e do modelo BM25 previamente definidos

# Definição de uma consulta que não está coberta pelos documentos
consulta = "Qual é a capital da Irlanda?"

# Tokenização da consulta
tokenized_query = word_tokenize(consulta.lower())

# Obtenção das pontuações BM25 para a consulta
doc_scores = bm25.get_scores(tokenized_query)

# Verificação se algum documento relevante foi recuperado
if max(doc_scores) == 0:
    print("Nenhum documento relevante foi recuperado.")
    top_n = []
else:
    # Obtenção dos documentos mais relevantes
    top_n = bm25.get_top_n(tokenized_query, documents, n=top_k)
    # Exibição dos documentos recuperados
    print("Documentos Recuperados:")
    for doc in top_n:
        print(f"- {doc}")


Documentos Recuperados:
- O aprendizado de máquina é uma subárea da inteligência artificial focada em algoritmos.
- A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas.
- A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.


In [11]:
# Uso do modelo de linguagem para gerar a resposta, mesmo sem contexto

# Se não houver documentos recuperados, o contexto será vazio
contexto = " ".join(top_n)

# Construção do prompt para o modelo de linguagem
prompt = f"Contexto: {contexto}\nPergunta: {consulta}\nResposta:"

# Geração da resposta
resposta = generator(prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=generator.tokenizer.eos_token_id)

# Exibição da resposta
print("\nResposta Gerada:")
print(resposta[0]['generated_text'])



Resposta Gerada:
Contexto: O aprendizado de máquina é uma subárea da inteligência artificial focada em algoritmos. A engenharia de prompt é crucial para orientar modelos de linguagem a produzirem respostas desejadas. A recuperação de informação trata de obter informações relevantes a partir de grandes conjuntos de dados.
Pergunta: Qual é a capital da Irlanda?
Resposta: It is a real life form that is not limited to the social and political elements. It also helps to facilitate exchange between the people, rather than the bureaucracy. I personally prefer to keep it natural and human, but the main goal is to make a


Análise:

Observação da Alucinação:
O modelo tenta responder à pergunta sem informações relevantes, possivelmente fornecendo uma resposta incorreta ou inventada.
Isso demonstra a limitação dos sistemas RAG convencionais em garantir respostas precisas quando a recuperação falha.